In [1]:
import pandas as pd
import json
from pprint import pprint
from collections import defaultdict

In [2]:
with open('./states_2010.json') as f:
    data = json.load(f)

In [3]:
csv_data = pd.read_csv(
    './StateData.csv',
    error_bad_lines=False
)
df = pd.DataFrame(csv_data)

In [4]:
# Note geojson file includes DC & Puerto Rico
dataset = data['features']
len(dataset)

52

In [5]:
# state_names = []
errors = []
# for state in dataset:
#     state_names.append(state['properties']['NAME'].lower())
state_code_key = defaultdict()

all_taxes = []
all_taxes_by_total = []
all_taxes_per_capita = []

# Refactor this for loop with a helper function to take a single column name (instead of writing the column name 4 times each time)

for state in dataset:
    try:
        df_state = df[(df['Name'].str.lower() == state['properties']['NAME'].lower()) & (df['Year4'] == 2016)]
#         print(state['properties']['STATE'])
#         print(int(df_state['State_Code']))
        state_code_key[int(state['properties']['STATE'])] = int(df_state['State_Code'])
        population = int(df_state.iloc[0]['Population'])
        total_revenue = int(df_state.iloc[0]['Total_Revenue'])
        total_expenditure = int(df_state.iloc[0]['Total_Expenditure'])
        correct_total_exp = int(df_state.iloc[0]['Correct_Total_Exp'])
        total_taxes = int(df_state.iloc[0]['Total_Taxes'])
        all_taxes.append(total_taxes)
        all_taxes_by_total.append(total_taxes/total_revenue)
        all_taxes_per_capita.append(total_taxes/population)
        state['properties']['population'] = population
        state['properties']['total_revenue'] = total_revenue
        state['properties']['total_expenditure'] = total_expenditure
        state['properties']['selected_field'] = total_taxes
        state['properties']['selected_field_as_fraction_of_total_rev'] = total_taxes/total_revenue
        
        state['properties']['selected_field_per_capita'] = total_taxes/population
        
    except:
        errors.append((df['Name'].str.lower()))
                      
                      
                      

In [45]:
dict(state_code_key)

{4: 3,
 5: 4,
 6: 5,
 8: 6,
 9: 7,
 13: 11,
 15: 12,
 17: 14,
 18: 15,
 22: 19,
 27: 24,
 28: 25,
 30: 27,
 35: 32,
 38: 35,
 40: 37,
 42: 39,
 47: 43,
 51: 47,
 10: 8,
 54: 49,
 55: 50,
 56: 51,
 1: 1,
 2: 2,
 12: 10,
 16: 13,
 20: 17,
 24: 21,
 34: 31,
 37: 34,
 45: 41,
 53: 48,
 50: 46,
 49: 45,
 19: 16,
 21: 18,
 23: 20,
 25: 22,
 26: 23,
 29: 26,
 31: 28,
 32: 29,
 33: 30,
 36: 33,
 39: 36,
 41: 38,
 44: 40,
 46: 42,
 48: 44}

In [14]:
max(all_taxes_per_capita)


4.929166094287604

In [15]:
min(all_taxes_per_capita)

1.4113202028081124

In [ ]:
# Max correct exp = 155231252
# Min correct exp = 1042164
# Max correct exp by total = .56
# Min correct exp by total = .13
# Max correct exp per capita = 4.93
# Min correct exp per capita = 1.4

In [48]:
errors

['puerto rico']

In [6]:
data['features'] = dataset
with open('./modified_states_2.json', 'w') as outfile:
    json.dump(data, outfile)